In [61]:
import os
from bs4 import BeautifulSoup

Для каждого отдельно взятого дела создаём словарь для метаданных, который постепенно заполняем:

In [43]:
meta_data = {'court': '', 'judge': '', 'prosecutor': '', 'secretary': '', 'accused': '', 'result': '', 'category': '', 
            'punishment_type': '', 'punishment_term': ''}

# category - это статья кодекса, остальное вроде ясно
# если что-то забыла -- фил фри добавить

In [53]:
def parse_xml(path):   # парсим xml
    with open(path, 'r', encoding='utf-8')as f:
        soup = BeautifulSoup(f.read(), 'html.parser')
        
        meta_data['court'] = soup.court.string     # добавляем в метаданные то, что имеется в разметке
        meta_data['judge'] = soup.judge.string
        meta_data['result'] = soup.result.string
        meta_data['category'] = soup.category.string
        
        html = []
        for line in soup.body:
            line = line.replace('[', '')
            line = line.replace(']', '') 
            html.append(line)
        true_html = ' '.join(html)
                
        html_soup = BeautifulSoup(true_html, 'html.parser')
        return html_soup.get_text()

In [57]:
def get_parts(text):    # делим на части
    lines = [line for line in text.split('\n')]
    beg, end = 0, 0 
    for num, line in enumerate(lines):
        if 'установил' in line.lower() or 'у с т а н о в и л' in line.lower(): 
            beg = num + 1
        if 'приговорил' in line.lower() or 'п р и г о в о р и л' in line.lower(): 
            end = num + 1
        if beg and end:
            beginning = [stroka.strip() for stroka in lines[:beg]]
            main_part = [stroka.strip() for stroka in lines[beg:end]]
            ending = [stroka.strip() for stroka in lines[end:]]
            return [' '.join(main_part), ' '.join(beginning), ' '.join(ending)]
        
        # для извлечения метаданных берём get_parts(text)[1:]
        # для всего остального - get_parts(text)[0]

In [58]:
parse_xml(r'C:\Users\a.reshetniko\Documents\trash\judgment\judgement\belinskij-rajonnyj-sud-penzenskaya-oblast-263\2010\100788109.xml')

{'court': 'Белинский районный суд (Пензенская область)', 'judge': 'Круглякова Букина Людмила Васильевна', 'prosecutor': '', 'secretary': '', 'accused': '', 'result': 'Обвинительный приговор', 'category': '158 ч.2 п.г; 158 ч.2 п.б', 'punishment_type': '', 'punishment_term': ''}


'\n\nП Р И Г О В О Р\nИменем Российской Федерации\n\n*** г. Белинский\nБелинский районный суд Пензенской области\nВ составе председательствующего Кругляковой Л.В.\nС участием государственного обвинителя – помощника прокурора Белинского района \nПензенской области Астафьевой Е.О.\nЗащитника Кердяшова С.М., представившего удостоверение № *** и ордер *** филиала *** областной коллегии адвокатов,\nПри секретаре Шебуровой В.И.\nВ присутствии подсудимого и гражданского ответчика Стульникова С.В.\nПотерпевшего и гражданского истца ФИО2,\nРассмотрев в открытом судебном заседании материалы уголовного дела в отношении\n\nСтульникова С.В. *** года рождения,\nуроженца села ***, \nгражданина РФ, со *** образованием, холостого, не \nработающего, проживающего без регистрации в *** \nрайона, ***, судимого:\n- приговором мирового судьи судебного участка № *** \nот *** по ч.1 ст. 112 УК РФ к 1 году л/свободы, \nосвобожденного по отбытии срока ***;\n- приговором *** районного суда от *** по ч.1\nст. 105 

In [59]:
meta_data

{'court': 'Белинский районный суд (Пензенская область)',
 'judge': 'Круглякова Букина Людмила Васильевна',
 'prosecutor': '',
 'secretary': '',
 'accused': '',
 'result': 'Обвинительный приговор',
 'category': '158 ч.2 п.г; 158 ч.2 п.б',
 'punishment_type': '',
 'punishment_term': ''}

In [60]:
find_main_part(parse_xml(r'C:\Users\a.reshetniko\Documents\trash\judgment\judgement\belinskij-rajonnyj-sud-penzenskaya-oblast-263\2010\100788109.xml'))

{'court': 'Белинский районный суд (Пензенская область)', 'judge': 'Круглякова Букина Людмила Васильевна', 'prosecutor': '', 'secretary': '', 'accused': '', 'result': 'Обвинительный приговор', 'category': '158 ч.2 п.г; 158 ч.2 п.б', 'punishment_type': '', 'punishment_term': ''}


(' Стульников С.В. совершил кражу, т.е. тайное хищение чужого имущества с незаконным проникновением в иное хранилище, при следующих обстоятельствах. В ночь на ***, в целях тайного хищения чужого имущества, действуя из корыстных побуждений, Стульников С.В. пришел к домовладению *** по *** в ***, принадлежащему ФИО2, где открыл рукой запорное устройство на входной двери и незаконно проник в сарай, расположенный на территории данного домовладения, откуда тайно похитил одного гуся стоимостью *** рублей, после чего с места преступления скрылся, распорядился похищенным имуществом по своему усмотрению, причинив ФИО2 материальный ущерб на сумму *** рублей. Он же, Стульников С.В., обвиняется в том, что ***, находясь в баре *** расположенном в *** в ***, в период времени с *** час. до *** часов, тайно похитил из кармана куртки ФИО4 сотовый телефон, стоимостью *** руб. *** коп., т.е. в совершении преступления, предусмотренного п. «г» ч. 2 ст. 158 УК РФ. В судебном заседании прокурор Астафьева Е.О